In [1]:
import pandas as pd
import os.path

root_path = os.path.dirname(os.getcwd())
text_raw = open(os.path.join(root_path, "DATA/list.txt"), "r").read()
text = '\n\n\n\n\n\n' + text_raw

In [18]:
sorta_sections = text.split('\n\n\n\n\n\n')[1:] # first item is empty
# sorta_titles_old = [sorta_section.strip().split('\n')[0] for sorta_section in sorta_sections] # all titles should be followed by three \n's
sorta_titles = [sorta_section.strip().split('\n\n\n')[0] for sorta_section in sorta_sections] # all titles should be followed by three \n's

# examine whether substring ending with either '(' or ',' is upper case
import re
# old_titles = [sorta_title for sorta_title in sorta_titles_old if re.split('\(|,', sorta_title)[0].isupper()]
titles = [sorta_title for sorta_title in sorta_titles if re.split('\(|,|and', sorta_title)[0].isupper()]
# new_titles = list(set(full_titles) - set(old_titles))
# titles = []
# for full_title in full_titles:
#     if full_title not in set(old_titles):
#         print(full_title)
#         titles.append(full_title)
# removed_titles = []
# for old_title in old_titles:
#     if old_title not in full_titles:
#         removed_titles.append(old_title)

In [5]:
section_starts = []
for title in titles:
    # how many newlines? I remember a catch..
    section_start = text.find('\n\n\n\n\n\n'+title+'\n\n\n')
    section_starts.append(section_start)

In [6]:
sections = []

# for index, section_start in zip(range(len(section_starts)), section_starts[:-1]):
for index, section_start in enumerate(section_starts[:-1]):
    section = text[section_start:section_starts[index+1]].strip()
    sections.append(section)
sections.append(text[section_starts[-1]:].strip())

In [9]:
def extract_subsections_data(section):
    data = pd.Series([])
    
    # sidebars can screw things up
    if '(See sidebar.)' in section:
        section = section[:section.find('(See sidebar.)')].strip()

    title = section.split('\n\n\n')[0].strip()
    data['name'] = title
    section = section[len(title):].strip()
    
    subsections = section.split('\n\n\n\n')
    subsections = [subsection.strip() for subsection in subsections]
    
#     if subsections and subsections[0] and subsections[0][0] == '[':
#         data['phonetic'] = subsections[0]
#         subsections = subsections[1:]
    # could be blank subsection after
    
#     if subsections:
#         for index, subsection in zip(range(len(subsections)), subsections):
    for subsection in subsections:      
        if subsection: # accounting for blank subsections, which I think exist?
#             if subsection[0] == '[':
#                 data['phonetic'] = subsection
#             elif 'Season' in subsection:
#                 data['season'] = subsection
#             elif 'Flavor' in subsection:
#                 data['flavor'] = subsection
#             elif 'Volume' in subsection:
#                 data['volume'] = subsection
#             elif 'Techniques' in subsection:
#                 data['techniques'] = subsection
#             elif 'Nutritional profile' in subsection:
#                 data['nutritional_profile'] = subsection
#             elif 'Calories' in subsection:
#                 data['Calories'] = subsection
#             elif 'Protein' in subsection:
#                 data['protein'] = subsection
#             elif 'Tip' in subsection:
#                 data['tip'] = subsection
#             elif 'Vegan Tip' in subsection:
#                 data['vegan_tip'] = subsection
#             elif 'Vegan substitute' in subsection:
#                 data['vegan_substitutes'] = subsection
#             elif 'Vegan Brand' in subsection:
#                 data['vegan_brands'] = subsection
#             elif 'Possible Substitute' in subsection or 'Possible substitute' in subsection: # both appear
#                 data['possible_substitutes'] = subsection
#             elif subsection[:5] == 'Brand': # was picking up "Brandy" in subsection
#                 data['brands'] = subsection
#             elif 'Botanical relatives' in subsection:
#                 data['botanical_relatives'] = subsection
#             elif 'Flavor Affinities' in subsection:
#                 data['flavor_affinities'] = subsection
#             elif 'What they are' in subsection:
#                 data['what_they_are'] = subsection
            if subsection.startswith('['):
                data['phonetic'] = subsection
            elif subsection.startswith('Season'):
                data['season'] = subsection
            elif subsection.startswith('Flavor') and not subsection.startswith('Flavor Affinities'):
                data['flavor'] = subsection
            elif subsection.startswith('Volume'):
                data['volume'] = subsection
            elif subsection.startswith('Techniques'):
                data['techniques'] = subsection
            elif subsection.startswith('Nutritional profile'):
                data['nutritional_profile'] = subsection
            elif subsection.startswith('Calories'):
                data['calories'] = subsection
            elif subsection.startswith('Protein'):
                data['protein'] = subsection
            elif subsection.startswith('Tip'):
                data['tip'] = subsection
            elif subsection.startswith('Vegan Tip'):
                data['vegan_tip'] = subsection
            elif subsection.startswith('Vegan substitute'):
                data['vegan_substitutes'] = subsection
            elif subsection.startswith('Vegan Brand'):
                data['vegan_brands'] = subsection
            elif subsection.startswith('Possible Substitute') or subsection.startswith('Possible substitute'): # both appear
                data['possible_substitutes'] = subsection
            elif subsection.startswith('Brand'): # was picking up "Brandy" in subsection
                data['brands'] = subsection
            elif subsection.startswith('Botanical relatives'):
                data['botanical_relatives'] = subsection
            elif subsection.startswith('Flavor Affinities'):
                data['flavor_affinities'] = subsection
            elif subsection.startswith('What they are'):
                data['what_they_are'] = subsection
            # checks if subsection consists of one or more single spaced lines
#             elif '\n\n' in subsection and not '\n\n\n' in subsection:
#                 data['pairs_with'] = subsection
    
    section_remainder = section
    
    # assuming 'Dishes' is last
    if 'Dishes' in section_remainder:
        dishes_start = section_remainder.find('Dishes')
        data['dishes'] = section[dishes_start:].strip()
        section_remainder = section_remainder[:dishes_start].strip() # removed to reveal pairs_with
    
    # removed to reveal pairs_with
    if 'Flavor Affinities' in section_remainder:
        section_remainder = section_remainder[:section_remainder.find('Flavor Affinities')].strip()
            
    # assumes pairs_with is always present, and followed only by dishes and/or flavor affinities
    # PROBLEM: sometimes quotes appear after pairs_with
#     data['pairs_with'] = section_remainder.split('\n\n\n')[-1].strip()
    
    subsections_remainder = section_remainder.split('\n\n\n')
    subsections_remainder = [subsection.strip() for subsection in subsections_remainder]
    
    
    for subsection in subsections_remainder:
        if '\n\n' in subsection and subsection[0] not in ['"', '“']: # check for single spaced line and not quote
            data['pairs_with'] = subsection
            break
        
        
#     i = 0;
#     for subsection in subsections:
#         data[str(i)] = subsection
#         i += 1
    return data

In [10]:
subsections_data = pd.Series(sections).apply(extract_subsections_data)

In [14]:
# subsections_data.to_csv(os.path.join(root_path, 'DATA/subsections_data_raw.csv'), index=False, header=None)

In [15]:
subsections_data.to_csv(os.path.join(root_path, 'DATA/subsection_data_header.csv'), index=False)

In [16]:
len(subsections_data['ingredient'])

822